We want to calculate the VaR (on the arithmetic variation of price, at a one-day horizon) for a call 
option on the Natixis stock. We will implement a Monte-Carlo VaR since the call price is a non-linear 
function of the underlying price, that we are able to model thanks to historical data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('Natixis_ihab.xlsx', header=None, names=['Date', 'Price'])

In [3]:
df.head()

,Date,Price
0,2015-01-02,5.621
1,2015-01-05,5.424
2,2015-01-06,5.329
3,2015-01-07,5.224
4,2015-01-08,5.453


We will compute the logReturn which is log(St/St-1)

In [8]:
df.set_index('Date', inplace=True)

In [9]:
df.loc[:,'Log_Return'] = np.log(df.loc[:,'Price']/df.loc[:,'Price'].shift(1))

In [10]:
df.head()

,Price,Log_Return
Date,,
2015-01-02,5.621,NaN
2015-01-05,5.424,-0.035676
2015-01-06,5.329,-0.017670
2015-01-07,5.224,-0.019900
2015-01-08,5.453,0.042903


In [11]:
df.tail()

,Price,Log_Return
Date,,
2018-12-21,4.045,-0.001482
2018-12-24,4.010,-0.008690
2018-12-27,3.938,-0.018118
2018-12-28,4.088,0.037383
2018-12-31,4.119,0.007555


On va calculer le lissage car on va porter moins d'importance aux données lointaines

In [27]:
lissage = 0.6
df.loc["2015-01-05", "Returns_lissés"] = df.loc["2015-01-05", "Log_Return"]
df.loc["2015-01-06":, "Returns_lissés"] = df.loc["2015-01-06":,"Log_Return"]*lissage + df.loc["2015-01-06":,"Log_Return"].shift(-1)*(1-lissage)

In [28]:
df.head()

,Price,Log_Return,Returns_lissés
Date,,,
2015-01-02,5.621,NaN,NaN
2015-01-05,5.424,-0.035676,-0.035676
2015-01-06,5.329,-0.017670,-0.018562
2015-01-07,5.224,-0.019900,0.005221
2015-01-08,5.453,0.042903,0.017365


Now we're going to simulate a number N (say N=1000 or else, but justify your choice for this number) of prices of 
the stock in a one-day horizon (we are working at the last date of 2018)
meaning that S0 = df.loc[:,'Price'].iloc(-1)
Before we have to estimate the mean and the variance of the brownian motion

In [48]:
S0 = df["Price"].iloc[-1]  # The last observed price in your dataset
log_returns = df.loc[:,"Returns_lissés"].dropna()
mumoinssigmasur2 = log_returns.mean() #car l'esperance de Ln(St/St-1)=mu - sigma*sigma*1/2
sigma = log_returns.std()
mumoinssigmasur2,sigma

(-0.0003020214761685973, 0.015119188513241465)

In [44]:
# Number of future days to simulate
n_days = 1000

# Generate Brownian motion: Random normal values for each day
brownian_motion = np.random.normal(0, 1, n_days)

# Calculate the future prices using the formula
future_prices = [S0]
for i in range(1, n_days):
    St = future_prices[i-1] * np.exp(mumoinssigmasur2 + sigma * brownian_motion[i-1])
    future_prices.append(St)

# Create a date range starting from the next day
future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=n_days, freq='D')

# Create a DataFrame with the future prices
future_df = pd.DataFrame(data=future_prices, index=future_dates, columns=["Estimated_Price"])

In [45]:
future_df.head()

,Estimated_Price
2019-01-01,4.119000
2019-01-02,4.173340
2019-01-03,4.200379
2019-01-04,4.244683
2019-01-05,4.206857


In [46]:
future_df["Estimated_Price"].mean()

3.70809023097896

Now let's calculate the price of the call
The price of the call is given by the black scholes formula
Ct = St*Phi(d1) -K*Phi(d2) ou Phi est la fonction de répartition d'une loi gaussienne centrée réduite
d1 et d2 sont des données à calculer

T = 22 (one month maturity) et t=1 (one day) 
meaning that T-t=21
K = S0 = 4,119

In [50]:
K = S0
r = 0
T_minus_t =21
future_df['d1'] = (np.log(future_df['Estimated_Price'] / K) + (r + 0.5 * sigma**2) * T_minus_t) / (sigma * np.sqrt(T_minus_t))
future_df['d2'] = future_df['d1'] - sigma * np.sqrt(T_minus_t)

In [51]:
future_df.head()

,Estimated_Price,d1,d2
2019-01-01,4.119000,0.034642,-0.034642
2019-01-02,4.173340,0.223807,0.154522
2019-01-03,4.200379,0.317017,0.247732
2019-01-04,4.244683,0.468457,0.399172
2019-01-05,4.206857,0.339259,0.269974


Now we're going to calculate the call price 

In [53]:
from scipy.stats import norm
future_df['Call_Price'] = future_df['Estimated_Price']*norm.cdf(future_df['d1']) - K*np.exp(-r * T_minus_t)*norm.cdf(future_df['d2'])

In [54]:
future_df.head()

,Estimated_Price,d1,d2,Call_Price
2019-01-01,4.119000,0.034642,-0.034642,0.113829
2019-01-02,4.173340,0.223807,0.154522,0.143793
2019-01-03,4.200379,0.317017,0.247732,0.160194
2019-01-04,4.244683,0.468457,0.399172,0.189109
2019-01-05,4.206857,0.339259,0.269974,0.164265


Now we can easily calculate the VaR using the empirical VaR (empirical quatile of these call prices)

In [55]:
confidence_level = 0.95

In [58]:
def empirical_var(returns, confidence_level):
    returns = returns.sort_values(ascending = False)
    return -np.percentile(returns, (1 - confidence_level) * 100)

In [62]:
VaR = empirical_var(future_df.loc[:,'Call_Price'],confidence_level)
print(f"\nHistorical VaR at {confidence_level * 100}% confidence level: {VaR}")


Historical VaR at 95.0% confidence level: -1.3295589326003506e-12


In [63]:
price_changes = future_df['Call_Price'].diff()
VaR2 = empirical_var(price_changes.dropna(), confidence_level)
print(f"\nHistorical VaR at {confidence_level * 100}% confidence level: {VaR2}")


Historical VaR at 95.0% confidence level: 0.06008285066384538
